In [1]:
#load libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Análisis de Datos con Pandas
### Virginia Domínguex García - Estación Biológica de Doñana - CSIC

Pandas es la herramienta de Python para trabajr con archivos csv, xlsx y demás bases de datos "cuadradas". Funciona de una forma similar a las dataframes de R, pero con algunas diferencias. 
Aquí podéis encontrar un chuletero con todas las cosas a recordar:
https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

**Tidy-Data:** En general Pandas mantiene la filosofía de tidy data, cada variables aparece en una **columna** y cada observación en una **fila**. Pandas automáticamente hace las operaciones vectoriales automáticamente, manteniendo este orden. Cada columna del DataFrame (pd.DataFrame) es una Serie (pd.Series), similar a un array con nombre en R. 

Así, las DataFrames son un conjunto de datos indexados, donde el índice de la fila me indica (en principio) una observación, y la columna me indica (en principio) la variable, aunque como veremos después, son bastante más versátiles gracias a los índices múltiles.

## 1 - Creando un Dataframe

Podemos crear un DataFrame desde 0 metiendo los datos y los índices (recuerda, el horizontal se llama **index** y el vertical **columns**):

In [ ]:
df = pd.DataFrame(
{"Sample" : ["R100" , "R201", "R203", "R340", "R453"],
"t0" : [0.2, 0.1, 0.3, 0.25, 0.13],
"t1" : [1.3, 1.8, 0.8, 1.5, 0.6],
"t2" : [2.8, 3.1, 1.9, 2.3, 1.8],
"t3" : [3.2, 3.7, 2.3, 3.5, 2.5],
"t4" : [1.2, 1.8, 3.9, 1.3, 3.7],
"t5" : [0.7, 0.4, 3.4, 0.3, 3.6]}
)
print(df)

In [ ]:
print(df.columns)
print(df.index)

In [ ]:
#Si nos quedamos con una columna tendremos una SERIE
type(df["t0"])

Aunque lo más interesante es importar una base de datos existente. Podemos usar archivos tanto csv (texto plano), como importar hojas de excel xlsx.

In [ ]:
filename="./Data/pokemon.csv"
pokemon_df=pd.read_csv(filename) 
pokemon_df.head(10)

Por defecto pandas usa la primera fila como el nombre de las columnas, pero introduce una nueva columna a la izquierda como índice. Si queremos que use alguna de las columnas como índice se lo podemos explicitar, y también podemos explicitar cuál fila queremos que use para dar nombre a las columnas:

In [ ]:
pokemon_df=pd.read_csv(filename, index_col=0,header=0) 
pokemon_df.head(10)

También podemos leer archivos excel:

In [ ]:
filename="./Data/SampleData.xlsx"
Sales_df=pd.read_excel(filename, sheet_name = "SalesOrders")
Sales_df.head(10)

Si queréis saber más: https://pythonbasics.org/read-excel/

In [ ]:
#ojo al leer,  verificar que el fichero que leemos tiene la estructura que pensamos. Este por ejemplo tiene muchas líneas arriba que son raras y no nos valen
filename="./Data/Pandas_ World Alcohol Consumption Dataset.csv"
alcohol_df=pd.read_csv(filename) #siempre revisar lo que vamos a leer skiprows=4

In [ ]:
filename="./Data/Pandas_ World Alcohol Consumption Dataset.csv"
alcohol_df=pd.read_csv(filename, skiprows=8) # or simply edit the original file
alcohol_df

## 2 - Data Overview  

Lo primero que nos va a interesar es tner una visión global de los datos, asegurarnos de que hemos leído el fichero correcto etc. 

Para ver la parte inicial o final del dataframe como hemos hecho antes, usaremos los métodos **.head()** and **.tail()**:

In [ ]:
pokemon_df.head(5)
#pokemon_df.tail(5)

y para saber el tamaño **.shape**, que nos devuelve un **tuple** con el número de filas y el número de columnas

In [ ]:
#para saber el tamaño, devuelve el numero de filas (observaciones) y de columnas (variable)
pokemon_df.shape

Para tener una idea general del tipo de dato que va en cada columna usamos **.info()**

In [ ]:
pokemon_df.info()

para tener un resumen estadístico usaremos el método **.describe()**, que nos da el numero de entradas, la media etc etc

In [ ]:
pokemon_df.describe()

Y podemos tener una visual rápida de cómo se distribuyen los valores de las columnas **numéricas** con **.hist()**

In [ ]:
pokemon_df.hist()
#esto es para que los ejes no se solapen
plt.tight_layout()
#imprime la imagen
plt.show()

Para las variables categóricas tenemos que hacer antes el conteo de casos con **.value_counts()**

In [ ]:
pokemon_df["Type 1"].value_counts().plot.bar()

#### * Comprobar si hay **variables correlacionadas**

In [ ]:
#  ver rápidamente cuáles están altamente correlacionadas por si tenemos que quitar alguna del análisis
pokemon_df.corr(method="spearman",numeric_only=True) #methods: ‘pearson’, ‘kendall’, ‘spearman’

In [ ]:
#y si lo queréis ver en colores, podemos hacer un heatmap (luego veremos otras representaciones)
sns.heatmap(pokemon_df.corr(method="spearman",numeric_only=True), cmap="coolwarm", annot=True)

Con esto deberíamos tener cubierto cómo hacernos una idea general de los datos que tenemos para empezar con la siguiente fase ...
Pero antes podéis intentar hacer unos ejercicios:

## 3 - Data preparation

He intentado hacer un resumen de los problemas más frecuentes que nos encontramos cuando vamos a analizar unos datos (pero si se os ocurren otros podemos intentar ver cómo solucionarlos).

### 3.1 **Selecionar** parte de la tabla: por **valores** de las celdas, por **índices**, o por **tipos** de datos
Cuando trabajamos con las DataFrames vamos a poder seleccionar filas, columnas, o celdas de **TRES FORMAS DIFERENTES**: usando los **valores** de las celdas (llamado selección booleana), usando los **índices** (el valor del índice y de la columna) y mediante el **tipo** de dato. Todas son muy útiles, así que es bueno conocerlas.

#### * Selección basada en el valor de las celdas: **Selecciones BOOLEANAS**
Vamos a seleccionar filas, columnas y celdas basándonos en el valor de los datos. Este tipo de selección se hace en dos partes: en la primera crearemos un array booleano que nos dice qué datos satisfacen la condición que buscamos y cuáles no, y una segunda parte en la que filtramos los datos usando ese array.

 

1 Condición: Por ejemplo, localicemoslos pokemons cuyo tipo 1 es Fuego:

In [ ]:
seleccion=pokemon_df["Type 1"] == "Fire"
pokemon_fuego_df=pokemon_df[seleccion]

Múltiples condiciones: Podemos combinar más de una condición con los operadores lógicos & (and) y | (or) y el de inclusión **(in)**.

Busquemos los pokemons de fuego de tercera generación:

In [ ]:
seleccion= (pokemon_df["Type 1"]=="Fire") & (pokemon_df["Generation"]==3)
pokemon_df[seleccion]

O los pokemons de Agua o fuego que sean legendarios:

In [ ]:
seleccion1 = (pokemon_df["Type 1"] == "Fire") | (pokemon_df["Type 1"]=="Water") 
# ojo que la columna Legendary YA ES UN BOOL!
pokemon_df[seleccion1 & pokemon_df["Legendary"]]   

In [ ]:
#pokemons de tipo Hierba o Fuego
seleccion1= pokemon_df["Type 1"].isin(["Grass","Ice"])  
display(pokemon_df[seleccion1])
#pokemons que empiezan con Gr
seleccion1= pokemon_df["Type 1"].str.startswith("Gr") #.str.contains('on')
pokemon_df[seleccion1]

Selecciones complicadas: **Query** (esta forma sólo permite lectura, es decir que podemos leer los datos pero NO cambiarlos)

In [ ]:
pokemon_df.query("Attack > Defense")
pokemon_df.query("Attack + Defense < 200")

#### * Selección basada en **tipo de columna**:

En algunos casos nos puede interesar quedarnos solo con las columnas que contengan un tipo de dato en particular (por ejemplo las columnas con datos numéricos). En este caso tenemos que usar los métodos que nos indiquen si las columnas son del tipo que nos interesan o no (es similar a la selección booleana que acabamos de ver, pero ahora se aplica al tipo de dato de cada columna).

Quedémonos sólo con las varibales numéricas de nuestra df de pokemons:

In [ ]:
pokemon_df.select_dtypes('number') #other possubilities are object, bool

si queremos quedarnos solo con los nombres de las columnas, los extraemos con el método .columns

In [ ]:
numeric_columns=pokemon_df.select_dtypes('number').columns
numeric_columns

#### * Selección basada en los **índices**: (.loc[], .iloc[])

Como acabamos de ver, este array de nombres de columnas es especial, pone **INDEX**. Esto es que son los índices de nuestros datos. Indexar correctamente l informción nos puede servir para encontrar la información más rápidamente. 

La forma más evidente es seleccionar sólo algunas columnas que nos interesen. Por ejemplo, si sólo quiero quedarme con el tipo 1 y el ataque 1 de los pokemos, puedo hacer este subset:

In [ ]:
pokemon_df.loc[:, ["Name","Type 1","Attack"]]
#como la suselección de columnas es muy frecuente también se puede usar esta forma + corta
#pokemon_df[["Name","Type 1","Attack"]] 
#aunque es más correcta la primera porque nos asegura no borrar datos de la df original sin querer

También podemos seleccionar en función del índice "horizontal", en este caso el número del poquemon. Encontremos por ejemplos los pokemon  con índice 3. Los dos puntitos en la derecha significan "selecciona todas las columnas".

In [ ]:
pokemon_df.loc[3,:]

En algunos casos nos puede interesar localizar las entradas por su orden en la base de datos, y para eso usaremos el método **.iloc[]**.

Por ejemplo, encontremos los pokemons que están en las filas 3 a 13, y las columnas 0, 5 y 8:

In [ ]:
pokemon_df.iloc[2:13,[0,5,8]]
#np.r_[1:5, 7:11, 13:15] if you want more complicated selections

#### **Multiindex:** indexar en filas y columnas usando múltples variables

Aunque pude parecer confuso, el uso de índices es una de las funcionalidades más importantes en pandas. Además, al contrario que R, ¡permite usar más de un nivel de indexado para las filas y para las solumnas!. Esto es una gran ventaja por ejemplo para poder gestionar bases de datos complejas, que queramos poder separar o combinar a nuestro antojo. 

Un **índice jerárquico** significa que la DataFrame tendrá dos o más dimensiones que se pueden usar para identificar cada fila. Para crear un MultiIndex con nuestro DataFrame original, pasamos una lista de columnas al método **.set_index()**

In [ ]:
filename="./Data/WordsByCharacter.csv"
lotr_df=pd.read_csv(filename)
lotr_df.head(5)

In [ ]:
multi_df = lotr_df.set_index(['Film', 'Chapter', 'Race', 'Character'])
multi_df.head(10)

Para evitar que los niveles aparezcan desordenados, usar  **.sort_index()** 

In [ ]:
multi_df = lotr_df.set_index(['Film', 'Chapter', 'Race', 'Character']).sort_index()
multi_df.head(10)

multi_df se ha organizado de modo que ahora hay cuatro columnas que componen el índice. Podemos verificarlo sacando el índice accedienco a **.index** o **index.values** (son tuples!!)

In [ ]:
#multi_df.index.values

Podemos reordenar los niveles como queramos:

In [ ]:
multi_df.reorder_levels(['Race','Character','Film','Chapter']).sort_index()

Y si queremos volver a como estaba antes y no usar índices, pues reseteamos los índices con **.reset_index()**

In [ ]:
multi_df.reset_index()#inplace=True

Podemos hacer **subsets usando los indices** múltiples en lugar de con selecciones booleanas:

Con **.loc[]** : Tenemos que **indicar el tuple entero**, permite editar

In [ ]:
multi_df.loc[('The Two Towers',slice(None),slice(None),['Gandalf','Saruman']), :]
#multi_df.loc[('The Two Towers',slice(None),slice(None),['Gandalf','Saruman']), :]=5
#multi_df.loc[('The Two Towers',slice(None),slice(None),['Gandalf','Saruman']), :]
#multi_df = lotr_df.set_index(['Film', 'Chapter', 'Race', 'Character']).sort_index()

Para esto es conveniente sabe **extraer los diferentes valores de cada uno de los índices**:

In [ ]:
level_names=multi_df.index.names
print(level_names)
multi_df.index.unique(level='Race')

Con **.xs()**: Return **cross-section** from the Series/DataFrame. Más sencillo, pero de sólo lectura (indicamos el valor del índice y el nombre del nivel). Es similar a lo que es .query() a la seleccion booleana.

In [ ]:
multi_df.xs('Galadriel', level='Character')
#multi_df.xs('Galadriel', level='Character')=5
#multi_df.loc[(slice(None),slice(None),slice(None),[''Galadriel']), :]=5
#multi_df.xs('Galadriel', level='Character')
#multi_df = lotr_df.set_index(['Film', 'Chapter', 'Race', 'Character']).sort_index()

Here str operators are a little more difficult, we need to operate on the index level values and then filter

In [ ]:
multi_df[multi_df.index.get_level_values('Character').str.contains('m')]

### 3.2 Limpieza de una base de datos

#### * **NAN**: gestionar información que no tenemos. 
Encontramos los NaN usando los métodos **is.na()** junto con seleccion booleana. 

In [ ]:
#encontrar columnas que tengan algún dato perdido
print(pokemon_df.isna().any())
#encontrar filas que tengan algún dato perdido
pokemon_df.isna().any(axis=1)

#ambas generan un vector boolean: True/False

In [ ]:
#contar los NaN en una columna en particular
print(pokemon_df['Type 2'].isnull().values.sum())

In [ ]:
#encontrar todas las FILAS (observaciones) que contengan algún NAN
pokemon_df[pokemon_df.isna().any(axis=1)]

In [ ]:
#encontrar todas las COLUMNAS (observaciones) que contengan algún NAN
pokemon_df.columns[pokemon_df.isna().any()].to_list()

Ahora podemos querer eliminar estas entradas usando **.dropna()** 

In [ ]:
#.dropna()
# Drop rows that has at least one NaN Values
clean_df=pokemon_df.dropna()
print(clean_df.head(5))
# Drop columns that has at least one Nan Values
clean_df=pokemon_df.dropna(axis=1)
print(clean_df.head(5))

#Otras que pueden ser de interes
# Drop rows that has NaN values on selected columns
df2=pokemon_df.dropna(subset=['Type 1','Type 2'])
# Drop rows that has all Nan Values
df=pokemon_df.dropna(how='all')
# Keep only the rows with at least 2 non-NA values.
df2=pokemon_df.dropna(thresh=3,axis=1)


o rellenar los NaN con algún valor concreto usando **.fillna()**

In [ ]:
pokemon_df['Type 2'].fillna(pokemon_df['Type 1'], inplace=True) #fill NaN values in Type2 with corresponding values of Type


#Otras modificaciones de interés
df_result = pokemon_df.fillna(value={'Type 2': 0, 'Legendary': 1}) #usar valores específicos para cada columna

#### * **Renombrar**  índices, columnas, o valores dentro de la base de datos: 

In [ ]:
#Podemos siempre crear un índice nuevo desde cero con rest_index:
print(pokemon_df.head(5)) #VEMOS QUE ES EL NÚMERO DE POKEMON, Y COMO ESTÁ MAL REPETIDO PUEDE SER UN MAL ÍNDICE!
pokemon_df.reset_index(inplace=True)
print(pokemon_df.head(5))

Podemos cambiar nombres de columnas y valores dentro de la dataframe. Por ejemplo, el nombre de los poquemons mega está raro. Podemos corregirlo usando **expresiones regulares** para quitar  todo lo que hay delante de "Mega" (más sobre expresiones regulares: https://dl.icewarp.com/online_help/203030104.htm , ojalá no os haga falta nunca)

In [ ]:
pokemon_df["Name"]=pokemon_df["Name"].str.replace(".*(?=Mega)", "",regex=True)

**Usando Dict**: En ocasiones es útil usar un diccionario de los que vimos antes, en los que pondremos como "keys" los valores a cambiar y como "value" el nuevo valor. Veamos un ejemplo

In [ ]:
#Cambiaemos algunos de los nombres de las columnas. Primero creamos el diccionario para mapear los valores antiguos y los nuevos
col_dict={"Attack": "Attack1", "Defense": "Defense1", "Sp. Atk":"Attack2", "Sp. Def": "Defense2"}
pokemon_df.rename(columns=col_dict) 

Esto por defecto genera una **NUEVA dataframe** (todo lo que se ejecute y se vea el ouput significa que genera una nueva). 
Para cambiarlo en la misma base de datos debemos indicarle **"inplace=True"**, o crear una nueva asignándola con "=".

In [ ]:
pokemon_df.rename(columns=col_dict, inplace=True) 
pokemon_df.head(5)

In [ ]:
#veamos si encontramos errores en los tipos de pokemon
np.sort(pokemon_df["Type 1"].unique())
#vemos que hay errores: Grasss, Normall,Poisson, podemos corregirlos de una forma similar

In [ ]:
correction_dict={"Grasss": "Grass", "Normall":"Normal", "Poisson": "Poison"}
pokemon_df["Type 1"].replace(correction_dict, inplace=True)
print(np.sort(pokemon_df["Type 1"].unique()))

A veces es importante saber si tienes duplicados y es pesado buscarlos a mano, podemos usar fuzzy logic (libreria FuzzyWuzzy). Por ejemplo encontrando duplicados en una columna. Para encontrar duplicados de lineas completas: https://maxhalford.github.io/blog/transitive-duplicates/

In [ ]:
from fuzzywuzzy import fuzz
type2=pokemon_df["Type 2"].unique()
for i in range(len(type2)):
    for j in range(len(type2)):
        typei=type2[i]
        typej=type2[j]
        distance = fuzz.partial_ratio(str(typei), str(typej))
        #print (typei, typej,distance)
        if   ( (80 < distance) and (typei != typej)) :
                   print(typei,typej)

In [ ]:
pokemon_df["Type 2"].replace(correction_dict, inplace=True) #corregimos tb la segunda columna porque vemos que no hya problemas nuevos
#podemos ver el histograma de valores con .value_counts()
pokemon_df["Type 2"].value_counts()

In [ ]:
# SELECCIÓN
## usando valores de filas y columnas (selección booleana)
## usando los tipos de las columnas df.select_dtypes(include=np.number)  ## seleccion de columnas numéricas
## usando los índices (uso de índices múltiples: MULTIINDEX)
### selección de columnas
### selección de filas
## busquedas complicadas: query

### 3.3 Operaciones con columnas y filas. Generando y guardando nueva información

Pandas hará todas las **operaciones vectoriales por defecto**, aplicando los operadors a todas las filas (o columnas) de la base de datos. El orden de los ejes en los que se aplican es primero por filas (axis=0) y luego por columnas (axis=1). Si recordamos las operaciones aritméticas (+, -, \*, /, //,  %) veamos cómo funcionan

Encontremos la suma de los dos valores de ataque de los pokemons, y guardemoslo en una columna nueva

In [ ]:
filename="./Data/pokemon_clean.csv"
pokemon_df=pd.read_csv(filename, index_col=0) # cargamos el fichero
pokemon_df.set_index("Name",inplace=True) # indexamos por nombre, que no está repetido

In [ ]:
pokemon_df["Total_Atk"]=pokemon_df["Attack"]+pokemon_df["Sp. Atk"] #guardar la suma de ataques en nueva columna
print(pokemon_df.head(4))
pokemon_df["Violence Ratio"]=pokemon_df["Attack"]/pokemon_df["Defense"] #calcular el ratio de violencia y guardarlo en una columna nuevaÇ
print(pokemon_df.head(4))

In [ ]:
#funciona de forma similar para las columnas de strings
pokemon_df["Composed Type"]=pokemon_df["Type 1"]+"_"+pokemon_df["Type 2"] #crear una columna con el tipo compuesto
pokemon_df.head(4)

### 3.4 Agrupando información: Funciones aritméticas y **.groupby()**

Tenemos diferentes funciones que nos permiten conocer la estadística agregada básica de nuestra base de datos, ya las vimos antes por encima, pero veamos cómo funcionan  en más detalle.
Las funciones son: **.sum() .mean() .meadian() .std() .max() .min() .count()** (hay que restringirse a las columnas numéricas)

In [ ]:
display(pokemon_df.select_dtypes('number').mean(axis=0)) #por filas: axis=0

pokemon_df.select_dtypes('number').mean(axis=1) #por columnas : axis=1

Más que aplicarlo a toda la base de datos, suele ser interesante si podemos **agrupar los datos por categorías**, y aplicar esas funciones a cada uno de los grupos. Para esto está la función "group_by".

Por ejemplo veamos el ataque medio de cada tipo de pokemons:

In [ ]:
pokemon_df.groupby(by="Type 1").mean()["Total_Atk"].sort_values(ascending=False)

Tambien es posible hacer esto de forma VISUAL, usando las categorias para separar las representaciones: **sns.boxplot()**, **sns.stripplot()**, **sns.violinplot** y **sns.barplot()**. Seaborn te ahce los cálculos sobre la dispersión de cada categoría por su cuenta, vale con que le indiquemos cuál usar


(Para incluir anotaciones de significancia estadística: https://levelup.gitconnected.com/statistics-on-seaborn-plots-with-statannotations-2bfce0394c00)

In [ ]:
sns.boxplot(x = "Type 1", y = "Total_Atk",data=pokemon_df)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
#veamos si ha habdo un cambio en los HP de los pokemons normal vs legendario a través de las generaciones. Hue me permite separar cada categoria en otras varias
sns.boxplot(x = "Generation", y = "HP", hue= "Legendary", data=pokemon_df)
plt.show()

In [ ]:
#lo cual es útil para hacer gráficos de barras:
types_df=pokemon_df.groupby(["Type 1", "Legendary"]).count()["#"].reset_index().sort_values(by="#",ascending=False)
#display(types_df)
sns.barplot(x="Type 1", y="#", hue="Legendary",data=types_df)
plt.xticks(rotation = 90)
plt.show()

### Correlaciones:
Aparte de ver similitudes o diferencias entre grupos, también nos pueden interesar obtener las correlaciones entre diferentes valores. Para eso podemos usar **sns.regplot()** (regresion lineal), **sns.scatterplot()**, o como vimos antes **sns.heatmap(pd.corr())**.
Veamos si existe algún tipo de correlacion entre el valor de ataque y de defensa

In [ ]:
sns.scatterplot(data=pokemon_df, x="Attack", y="Defense",hue="Legendary")
#sns.regplot(data=pokemon_df, x="Attack", y="Defense")

O entre 

In [ ]:
fig, ax =  plt.subplots(1)
sns.regplot(data=pokemon_df, x="Sp. Atk", y="Sp. Def", ax=ax)
#para meter el numero hayq¡ que currarselo un poco
from Funciones import *
corr_func_annotate(pokemon_df["Sp. Atk"], pokemon_df["Sp. Def"], ax=ax, method="pearson", color='black',  xy=(0.6,0.8),fontsize=16)

### Ordenando informacion: **.sort()**
Estos métodos valen para ordenar una columna acorde a sus valores, en orden ascendente o descendente. Por ejemplo, encontremos los tres pokemons con los ataques más altos:

In [ ]:
pokemon_df.sort_values('Attack',ascending=False).head(3) # ordenamos por ataque y nos quedamos con los tres primeros

Podemos buscar directamente los poquemons con **valores extremos**:

In [ ]:
print("Max DEFENCE:",(pokemon_df['Defense']).idxmax()) #idxmax idxmin devuelve el índex label del valor máximo
print("Max HP value is %s, from %s" % (pokemon_df['HP'].max(), pokemon_df['HP'].idxmax()))

pokemon_df.loc[pokemon_df['Defense'].idxmax()]

### 3.5 Combinando DataFrames: pd.concat() y pd.merge()

En ocasiones es posible que te encuentres con los datos divididos entre más de un dataframe. 

**.concat()**: Si vamos construyendo la dataframe poco a poco necesitaremos esto. El método **pd.concat()** permite combinar todas las df que le pasemos como variable en una dataframe única. La lógica es similar a append

In [ ]:
# creating the DataFrames
df1 = pokemon_df.sample(5)
display('df1:', df1)
df2 = pokemon_df.sample(5)
display('df2:', df2)
  
# concatenar, por defecto concatenma por filas (index=0)
display('After concatenating:')
display(pd.concat([df1, df2]))#, keys = ['key1', 'key2'])) #esto permite retener la df original si es necesario

**.merge()**: Si tenemos los datos referentes a las mismas observaciones repartidos en más de 1 dataset. Esto permite hacer las uniones típicas: inner, outer, left, y rigth

In [ ]:
df1 = pd.DataFrame({   
        "key": ["K0", "K1", "K2", "K4"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    })
df2 = pd.DataFrame({
        "key": ["K0", "K1", "K2", "K3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    })

display('Outer join:', pd.merge(df1, df2, on="key",how='outer')) #'left', 'right', 'inner', 'outer'
display('Right join:(df2 + importante)', pd.merge(df1, df2, on="key",how='right'))
display('Left join:(df1 + importante)', pd.merge(df1, df2, on="key",how='left'))
display('Inner join:',pd.merge(df1, df2, on="key",how='inner'))

In [ ]:
# COMBINANDO DATAFRAMES
## concatenación
## uniones (inner, outer, left rigth)

### 3.6 Restructurando DataFrames: pd.merge() y pd.concat()

En muchos casos nos va a interesar pasar de tener los valores en filas a tenerlos repartidos en columnas (estirar la df), o combinar los valores de varias columnas en una (compactar la df) si queremos que nuestros datos tengan una estructura concreta https://www.w3resource.com/pandas/dataframe/dataframe-pivot.php

* Colapsar varias filas a una columna: **.melt()**

Imagina que tienes esta DataFrame con la lluvia de cada año en verano (RainW) e invierno (RainS). Para pasarla a formato tidy usaremos .metl():

In [ ]:
df = pd.DataFrame({'Year': {0: '14', 1: '15', 2: '16'},
                   'RainW': {0: 1, 1: 3, 2: 5},
                   'RainS': {0: 2, 1: 4, 2: 6}})
display(df)

In [ ]:
df.melt(id_vars=['Year'], value_vars=['RainW','RainS']) #B y C entran como indexes. Para convertir a tidy style:la cantidad de agua en función de diferentes indices

* Expandir una columna a varias filas: **.pivot()**

Imagina ahora que tienes una lsita de adyacencia de interacciones entre plantas y polinizadores como la de abajo, pero te interesa trabajar con la matriz de adyacencia. Puedes usar .Pivot para extender la columna de animales a diferentes columnas, y quedarnos con los valores de una de las interacciones (o las dos)

In [ ]:
#de datos en 1 clumna a varias columnas: pivot
df = pd.DataFrame({'Plant': ['P1', 'P1', 'P1', 'P2', 'P2',
                           'P2'],
                   'Animal': ['A1', 'A2', 'A3', 'A1', 'A2', 'A3'],
                   'Dispersion': [1, 3, 0, 0, 3, 1],
                   'Herbivory': [0, 2, 1, 1, 2, 0]})
display(df)

In [ ]:
incidence_M=df.pivot(index='Plant', columns='Animal', values='Dispersion') # incidence matrix of dispersion
display(incidence_M)
sns.heatmap(incidence_M, cmap="BuPu")
plt.show()
df.pivot_table(values=['Dispersion', 'Herbivory'], index='Plant', columns='Animal') # multi incidence matrix of dispersion and herbivory

In [ ]:
#siempre podemos intercambiar filas y columnas con .T, haciendo la traspuesta
df.T

In [ ]:
# RE-ESTRUCTURANDO LOS DATOS
## Pivot
## Traspuesta